# Test model preparation

In [1]:
!pip install trl peft bitsandbytes sentencepiece wandb
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 26.8 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [4]:
def get_model(model_path):
    # Load the base model with the same configuration as during training
    base_model = AutoModelForCausalLM.from_pretrained(
        "microsoft/Phi-3.5-mini-instruct",
        torch_dtype=torch.float16,
        load_in_4bit=True
    )
    base_model.config.use_cache = False
    
    # Load the PEFT model from the checkpoint directory
    test_model = PeftModel.from_pretrained(base_model, model_path)
    test_model = test_model.to("cuda")
    print(f'Model loaded from {model_path}')
    return test_model

# Dataset Preparation

In [5]:
import pandas as pd
import json
import re

In [6]:
lisa_sheets =  pd.read_csv("/kaggle/input/eval-data/lisa_sheets.csv")

with open('/kaggle/input/eval-data/test_folders.json', 'r') as file:
    test_folders = json.loads(file.read())

lisa_sheets = lisa_sheets[lisa_sheets['folder'].isin(test_folders)]

In [7]:
def extract_id_from_content(content):
    match = re.search(r"\|Identifiant=(.*?)\s*\|", content)
    return match.group(1) if match else None

In [8]:
lisa_sheets['id'] = lisa_sheets['content'].apply(extract_id_from_content)

In [9]:
def remove_double_bracketed_text(text):
    # Use regex to find and remove text within {{...}}
    cleaned_text = re.sub(r"\{\{.*?\}\}", "", text, flags=re.DOTALL)
    return cleaned_text.strip()

In [10]:
lisa_sheets['content'] = lisa_sheets['content'].apply(remove_double_bracketed_text)

In [11]:
lisa_sheets = lisa_sheets.drop('question', axis=1)

In [12]:
lisa_sheets = lisa_sheets.reset_index(drop=True)

# Formatter model preparation

In [13]:
!pip install langchain_openai
!pip install langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR

In [14]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from kaggle_secrets import UserSecretsClient

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
parser_model = ChatOpenAI(model="gpt-4o", temperature = 0.7, api_key = UserSecretsClient().get_secret("OPENAI_KEY"))

In [16]:
class MCQQuestion(BaseModel):
    question: str = Field(description="The multiple-choice question")
    option_a: str = Field(description="The first answer option labeled 'A'")
    option_b: str = Field(description="The second answer option labeled 'B'")
    option_c: str = Field(description="The third answer option labeled 'C'")
    option_d: str = Field(description="The fourth answer option labeled 'D'")
    correct_option: str = Field(description="This consists only a letter of correct option")

mcq_parser = JsonOutputParser(pydantic_object=MCQQuestion)

prompt_template = PromptTemplate(
    template="{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": mcq_parser.get_format_instructions()},
)

chain = prompt_template | parser_model | mcq_parser

# Functions

In [17]:
system_instruction = """
Based on the following educational content, generate a multiple-choice question with four answer options where only one is correct.
The question should assess understanding of the main ideas, and the options should be clear, informative, and relevant.
Ensure that the distractors (incorrect options) follow a logical but incorrect interpretation, based on common misconceptions or misunderstandings of the topic.
Ensure that the way the question is constructed does not allow a test taker with no relevant medical knowledge to identify the correct answer through clues in the phrasing, structure, answer choice formatting, or other linguistic hints.
Answer options must be as short as possible.

"""

def get_response_from_phi3(model, lisa_sheet):
    inputs = None
    outputs = None
    try:
        full_prompt = f"<|system|>{system_instruction}<|end|>\n\n<|user|>{lisa_sheet}<|end|>\n\n<|assistant|>"
        inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(inputs.input_ids, max_length=3584, temperature=0.7, do_sample=True)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    except Exception as error:
        print("An exception occurred:", error)
    finally:
        del inputs  # Free up GPU memory used by inputs
        del outputs  # Free up GPU memory used by outputs
        torch.cuda.empty_cache()

    return generated_text

In [18]:
def structurize_lisa_sheet_response(raw_response):
    return chain.invoke({"query": raw_response})

In [19]:
import shutil

def single_model_eval(model, model_name):
    if os.path.exists(f'/kaggle/working/{model_name}'):
        shutil.rmtree(f'/kaggle/working/{model_name}')
        
    os.mkdir(f'/kaggle/working/{model_name}')
    with open(f'/kaggle/working/{model_name}/responses.txt', 'w') as file:
        pass

    mcqs = []
    for index, row in lisa_sheets.iterrows():
        if index % 10 == 0:
            print(f'{model_name}, {index}, {datetime.datetime.now()}')
        try:
            lisa_sheet = row["content"]
            lisa_sheet_content = remove_double_bracketed_text(lisa_sheet)
            phi3_mcq = get_response_from_phi3(model, lisa_sheet_content)
            structured_mcq = structurize_lisa_sheet_response(phi3_mcq)
            structured_mcq["id"] = row["id"]
            compact_mcq = json.dumps(structured_mcq, separators=(',', ':'))
            mcqs.append(compact_mcq)
            with open(f'/kaggle/working/{model_name}/responses.txt', 'a') as file:
                file.write(f'{compact_mcq}\n')
        except Exception as error:
            print("An exception occurred:", error)
            with open(f'/kaggle/working/{model_name}/mcqs.json', 'w') as file:
                json.dump(mcqs, file)
    
    with open(f'/kaggle/working/{model_name}/mcqs.json', 'w') as file:
        json.dump(mcqs, file)

# Generation

In [20]:
import datetime
import os
import json

models_path = "/kaggle/input/finetuned-models"
for root, models, _ in os.walk(models_path):
    model = None
    for model_name in models:
        subfolder_path = os.path.join(root, model_name)
        try:
            model = get_model(subfolder_path)
            single_model_eval(model, model_name)
        except Exception as error:
            print(f"An exception occurred for {model_name}:", error)
        finally:
            del model
            torch.cuda.empty_cache()

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded from /kaggle/input/finetuned-models/new_model/checkpoint-20
checkpoint-20, 0, 2024-12-08 03:43:17.410280
checkpoint-20, 10, 2024-12-08 03:47:05.068923
checkpoint-20, 20, 2024-12-08 03:51:24.687543
checkpoint-20, 30, 2024-12-08 03:55:13.099504
checkpoint-20, 40, 2024-12-08 03:59:09.286203
checkpoint-20, 50, 2024-12-08 04:03:34.798436
checkpoint-20, 60, 2024-12-08 04:08:04.169157
checkpoint-20, 70, 2024-12-08 04:11:40.493360
checkpoint-20, 80, 2024-12-08 04:15:55.113369
checkpoint-20, 90, 2024-12-08 04:20:09.269566
checkpoint-20, 100, 2024-12-08 04:24:06.777979
checkpoint-20, 110, 2024-12-08 04:28:00.363470
checkpoint-20, 120, 2024-12-08 04:32:18.157200
checkpoint-20, 130, 2024-12-08 04:36:22.673934


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded from /kaggle/input/finetuned-models/new_model/best_model
best_model, 0, 2024-12-08 04:36:48.754596
best_model, 10, 2024-12-08 04:40:12.075171
best_model, 20, 2024-12-08 04:44:05.163150
best_model, 30, 2024-12-08 04:47:22.156851
best_model, 40, 2024-12-08 04:51:00.876639
best_model, 50, 2024-12-08 04:54:00.583587
best_model, 60, 2024-12-08 04:57:00.933830
best_model, 70, 2024-12-08 05:00:04.897196
best_model, 80, 2024-12-08 05:05:26.653978
best_model, 90, 2024-12-08 05:08:36.350698
best_model, 100, 2024-12-08 05:12:05.287611
best_model, 110, 2024-12-08 05:15:33.207630
best_model, 120, 2024-12-08 05:19:06.062600
best_model, 130, 2024-12-08 05:22:23.979112
